
# Jacobian for homogenous gas reactor

***Energy equation***
$$
\frac{dT}{dt}= -\frac{1}{\rho c_p}\sum_{k=1}^{N_{spec}}\dot{\omega_{k}} W_k h_k = S_T
$$

***Species equation***
$$
\frac{dY_k}{dt}=\frac{1}{\rho}\dot{\omega_{k}}W_k=S_{Y_k},\,\,\,k=1\ldots N_{spec}
$$

## Environment Setup

In [1]:
TChem_install_directory ='where/tchem/is/installed'

In [2]:
import sys
sys.path.append(TChem_install_directory+'/lib')
import pytchem
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
sys.path.append(TChem_install_directory+'/example/runs/scripts/')
import pmixSample

## Pre-simulation
* Set variables; temperature and stoichiometric ratio (fuel/air).
* Convert from stoichiometric ratio to mass fraction of CH4, O2, N2 and AR.
* Create samples spaning over the variable ranges.

In [4]:
# Pressure, Temperature, phi(stoichiometric ratio)
one_atm = 101325 # [Pa]
TempMax = 1300   # Maximum temperature [K]
TempMin = 1000   # Minimum temperature [K]

PressureMax = 1*one_atm; # [Pa]
PressureMin = 1*one_atm; # [Pa]

phiMax = 3.0; # Maximum equivalence ratio [-]
phiMin = 0.8; # Minimum equivalence ratio [-]

In [5]:
Npp = 1
Npt = 2 
Npphi = 2
N = Npp*Npt*Npphi

pressure    = [PressureMin] #
temperature = np.linspace(TempMin, TempMax, Npt)
eqratio     = np.linspace(phiMin, phiMax, Npphi)
p, temp, phi = np.meshgrid(pressure, temperature,eqratio)
p    = p.reshape(np.size(p))       #flatten()
temp = temp.reshape(np.size(temp)) #flatten()
phi  = phi.reshape(np.size(phi))   #flatten()

In [6]:
Nvar = 6
sample = np.zeros([N,Nvar])
fuel ="CH4"
nC=1

In [7]:
for i in range(N):
    sample[i,0] = temp[i]
    sample[i,1] = p[i]
    Yp_fuel, Yr_o2, Yr_n2, Yr_ar = pmixSample.getMassFraction(nC,phi[i])
    sample[i,2] = Yp_fuel
    sample[i,3] = Yr_o2
    sample[i,4] = Yr_n2 
    sample[i,5] = Yr_ar

## TChem Simulation

### Initialize TChem Driver Object

* Initialization of Kokkos.
* Create a TChem driver object. 

In [8]:
pytchem.initialize()

In [9]:
tchem = pytchem.TChemDriver()

* Get help from TChem driver object.

In [10]:
help(tchem)

Help on TChemDriver in module pytchem object:

class TChemDriver(pybind11_builtins.pybind11_object)
 |  A class to manage data movement between numpy to kokkos views in TChem::Driver object
 |  
 |  Method resolution order:
 |      TChemDriver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: pytchem.TChemDriver) -> None
 |  
 |  cloneGasKineticModel(...)
 |      cloneGasKineticModel(self: pytchem.TChemDriver) -> None
 |      
 |      Internally create clones of the kinetic model
 |  
 |  computeGasEnthapyMass(...)
 |      computeGasEnthapyMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute enthalpy mass and mixture enthalpy
 |  
 |  computeGasNetProductionRatePerMass(...)
 |      computeGasNetProductionRatePerMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute net production rate
 |  
 |  computeGasReactionRateConstants(...)
 |      computeGasReactionRateConstants(sel

### Create Kinetic Model 

* Inputs are the reactions mechanism files; in this case, we use the GRI3.0 gas reaction mechanism

In [11]:
inputs_directory = TChem_install_directory + '/example/data/ignition-zero-d/gri3.0/'
tchem.createGasKineticModel(inputs_directory+'chem.inp',inputs_directory+'therm.dat')

* Set number of samples
* Internally construct const object of the kinetic model and load them to device

In [12]:
tchem.setNumberOfSamples(N)
tchem.createGasKineticModelConstData()

### Set State Vector

* Get index for variables. 
* Pass a numpy array to the TChem object to set the state vector. 

In [13]:
Variables = ['T','P','CH4','O2','N2','AR']
indx=[]
for var in Variables:
    indx += [tchem.getStateVariableIndex(var)]

state = np.zeros([N, tchem.getLengthOfStateVector()])
for sp in range(N):
    state[sp,indx] = sample[sp,:]

In [14]:
tchem.createStateVector()
tchem.setStateVector(state)

In [15]:
tchem.computeJacobianHomogeneousGasReactor()

In [16]:
jacobian = tchem.getJacobianHomogeneousGasReactor(0)

In [17]:
print(np.shape(jacobian))
print(jacobian)

(54, 54)
[[-1.29675785e-03 -1.14169098e+01  1.62362331e+10 ... -3.65045856e+01
  -2.48517813e+08 -7.47820624e+02]
 [ 0.00000000e+00 -1.42542785e-04  3.85230404e+05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.81140419e-12  7.12720953e-05 -4.48862740e+05 ...  3.33532729e-09
   2.60203217e+03 -5.75883890e-12]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -5.89280062e-03
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -2.09116838e+05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -1.97077911e-01]]


In [18]:
# compute rhs of Homogeneous Gas Reactor
tchem.computeRHS_HomogeneousGasReactor()

In [19]:
# get RHS for sample 0
tchem.getRHS_HomogeneousGasReactor(0)

array([-4.75837752e-02,  0.00000000e+00,  9.70832134e-11,  6.78919068e-15,
       -8.32264203e-06,  0.00000000e+00,  0.00000000e+00,  8.58480727e-06,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.91195170e-06, -4.17421403e-06,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.26413468e-16,
        0.00000000e+00,  0.00000000e+00,  1.83880439e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.25979150e-14,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [20]:
del(tchem)

In [21]:
pytchem.finalize()